In [1]:
import pandas as pd
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch
from sklearn.model_selection import train_test_split
import torch.nn as nn
from torch import optim, Tensor
import torch.functional as F
import math
from tqdm.notebook import tqdm
from chords_dataset import ChordsDataset
from model_helpers import NLP, preprocess_text
from model import Transformer
device = torch.device("cpu" if torch.cuda.is_available() else "cp   u")

df = pd.read_csv('./model_data/lyrics_processed_2.csv')
dataset = ChordsDataset(df, NLP)
train_i, test_i, validation_i = dataset.get_train_test_valid_indexes(0.9,0.09,0.01)



C:\Users\bench\AppData\Roaming\Python\Python38\site-packages\spacy\util.py:837: UserWarning: [W095] Model 'en_core_web_sm' (3.2.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.3.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
c:\Users\bench\anaconda3\envs\p38\lib\site-packages\torchtext\data\utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')


In [2]:
import random
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        # self.embedding = nn.Embedding(input_size, hidden_size)
        self.inLinear = nn.Linear(input_size,hidden_size).to(device)
        self.out = nn.Linear(hidden_size,hidden_size).to(device)
        self.gru = nn.GRU(hidden_size, hidden_size).to(device)

    def forward(self, input, hidden):
        embedded = self.inLinear(input.view(1, 1, -1))
        output = nn.functional.relu(embedded)
        output, hidden = self.gru(output, hidden)
        hidden = self.out(hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
        
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size).to(device)
        self.gru = nn.GRU(hidden_size, hidden_size).to(device)
        # self.out = nn.Linear(hidden_size, output_size)
        self.out2 = nn.Linear(hidden_size,output_size).to(device)
        self.softmax = nn.LogSoftmax(dim=2).to(device)

    def forward(self, input, hidden):
        output = self.embedding(input.to(device)).view(1, 1, -1)
        output = nn.functional.relu(output).to(device)
        output, hidden = self.gru(output, hidden)
        # output = self.out(output)
        output = self.out2(nn.functional.relu(output).to(device))
        output = self.softmax(output)
        # print("Input of decoder after softmax  + sum",output.shape, output.sum())
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [3]:
teacher_forcing_ratio = 0.9


def train(input_tensor: Tensor, target_tensor: Tensor,rarity_tensor: Tensor,  encoder: nn.Module, decoder: nn.Module, encoder_optimizer, decoder_optimizer, criterion, max_length=7039):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = 4
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input =target_tensor[0].view(1,1).type(torch.LongTensor).to(device)

    decoder_hidden = encoder_hidden
    decoded = [decoder_input]

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(1,4):
            decoder_output, decoder_hidden  = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(
                decoder_output.view(1,-1,1).to(device),
                target_tensor[di].view(1,1).type(torch.LongTensor).to(device)
            )
            decoded.append(decoder_output.topk(1)[1])
            decoder_input = target_tensor[di].view(1,1).type(torch.LongTensor)    # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(1,4):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input
            decoded.append(topi)
            loss += criterion(
                decoder_output.view(1,-1,1).to(device),
                target_tensor[di].view(1,1).type(torch.LongTensor).to(device)
            )

    weighted_loss = loss * rarity_tensor.item()
    weighted_loss.backward()
    # print(dataset.decode_chord_tensor([d.item() for d in target_tensor]), dataset.decode_chord_tensor([d.item() for d in decoded]))
    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [4]:

import time

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def predictLyrics(lyrics : str) -> list:
    
    return ["COu", "Cou"]

def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for i,index in enumerate(train_i):
        elem = dataset[index]
        source, target, rarity = elem["lyrics"].reshape(-1,1,100).to(device), elem["chords"].reshape(-1,1).to(device), Tensor([elem["rarity"]]).reshape(-1,1).to(device)
        loss = train(source, target,rarity, encoder,
                    decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if i % print_every == 1:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print(timeSince(start, i / len(train_i)), (i / len(train_i)) * 100, print_loss_avg)

        if i % print_every == 1:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0
        # if iter % plot_every == 0:
        #     showPlot(plot_losses)

    showPlot(plot_losses)


In [5]:
hidden_size = 256
output_dim = len(dataset.chords_set)
encoder1 = EncoderRNN(100, hidden_size).to(device)
attn_decoder1 = DecoderRNN(hidden_size, output_dim).to(device)

trainIters(encoder1, attn_decoder1, len(dataset), print_every=75, learning_rate= 2e-4)

c:\Users\bench\Documents\EPFL\Master\MA4\Computador e musica\project\ComputadorMusica\chords_dataset.py:80: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:201.)
  "lyrics": Tensor([self.vectorize(word) for word in  self.lyrics[idx] ]),


0m 0s (- 368m 48s) 0.002837120889721111 0.1511724853515625
0m 17s (- 137m 8s) 0.21562118761880444 5.6371564865112305
0m 34s (- 135m 0s) 0.42840525434788773 3727.333355525335
0m 50s (- 129m 57s) 0.6411893210769711 4.444101371765137
1m 6s (- 129m 16s) 0.8539733878060544 4.426296459833781


KeyboardInterrupt: 

In [44]:
def evaluate(encoder, decoder, sentence, max_length=7039):
    with torch.no_grad():
        input_tensor = Tensor([dataset.vectorize(w) for w in preprocess_text( sentence)]).to(device)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)


        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(
                input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] = encoder_output[0, 0]
        decoder_input = torch.tensor([dataset.chord2id[dataset.draw_chord()[0]]], device=device)

        decoder_hidden = encoder_hidden

        decoded_words = [random.sample(dataset.chords_set,1)[0]]
        # decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(1,4):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            decoded_words.append(dataset.id2chord[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words
lyrics = '''
Today is gonna be the day that they're gonna throw it back to you
And by now, you should've somehow realised what you gotta do
I don't believe that anybody feels the way I do about you now
And backbeat, the word is on the street that the fire in your heart is out
I'm sure you've heard it all before, but you never really had a doubt
I don't believe that anybody feels the way I do about you now
And all the roads we have to walk are winding
And all the lights that lead us there are blinding
There are many things that I would like to say to you, but I don't know how
Because maybe

'''
evaluate(encoder1, attn_decoder1,lyrics, 1500 )

['Abm/Eb', 'Ab', 'Ab', 'Ab']

In [19]:
random.sample(dataset.chords_set,1)[0]

'Fsus4'